In [2]:
import ijson
import pandas as pd
import numpy as np
import re
import string
from langdetect import detect
import nltk
from stop_words import get_stop_words
from nltk.corpus import stopwords
import tqdm
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob


In [4]:
User = pd.read_csv('user_selected.csv')
Review_project = pd.read_csv('Review_project_sentiment.csv')

In [5]:
User["elite"][:100]

0     2007,2008,2009,2010,2011,2012,2013,2014,2015,2...
1                         2009,2010,2011,2012,2013,2014
2                                             2010,2011
3                         2010,2011,2012,2013,2014,2015
4                    2012,2013,2014,2015,2016,2017,2018
                            ...                        
95                        2009,2010,2012,2013,2014,2015
96    2010,2011,2012,2013,2014,2015,2016,2017,2018,2...
97                   2009,2010,2011,2012,2013,2014,2015
98                        2008,2009,2010,2011,2012,2013
99                                       2007,2008,2009
Name: elite, Length: 100, dtype: object

In [6]:
User["NumElite"] = User["elite"].str.count("200")+User["elite"].str.count("201")

In [7]:
User["NumElite"]

0        13.0
1         6.0
2         2.0
3         6.0
4         7.0
         ... 
39198     NaN
39199     NaN
39200     NaN
39201     NaN
39202     NaN
Name: NumElite, Length: 39203, dtype: float64

In [8]:
NewUser = User[["user_id","review_count","NumElite","average_stars"]]

In [9]:
NewUser

,user_id,review_count,NumElite,average_stars
0,dIIKEfOgo0KqUfGQvGikPg,2136,13.0,4.09
1,JnPIjvC0cmooNDfsa9BmXg,987,6.0,3.77
2,cojecOwQJpsYDxnjtgzteQ,51,2.0,3.79
3,1jXmzuIFKxTnEnR0pxO0Hg,299,6.0,3.43
4,-8QoOIfvwwxJ4sY201WP5A,288,7.0,3.88
...,...,...,...,...
39198,BFi55xLKztWtD_7cRl05EA,3,NaN,2.33
39199,p_0uPikwwHXbDsMy2_VtsQ,1,NaN,1.00
39200,dciX3Vn81ajo1zuDV3eRDg,1,NaN,2.33
39201,2LWgWsJbO8lwWJw4rURJ8Q,3,NaN,3.67


In [10]:
Review_project=pd.read_csv('Review_project_sentiment.csv')

In [11]:
Review_project

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,moderate useful,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,moderate useful,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,not useful,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111,0.218750
3,141509,OMw52-Eo-BYFHdUldLMc5Q,mmr-P0i__uQJHGXXeVjXxg,0Ng-wDNyA-96uaJZP8EKZg,4.0,40,0,0,When I first came here for the first time last...,2015-03-09 22:42:06,useful,en,first came first time last year best friend ap...,83.96,0.168452,0.419841,0.168831
4,7962637,RWq78T9FIyauVHaHj7yl9g,CyCI71vLDa923zOMeDDA5w,WUfZXjTtBUbezJE7LnWABw,3.0,0,0,0,"I only got the take-out, so I cannot speak of ...",2013-03-22 20:47:40,not useful,en,got takeout cannot speak service food good wou...,51.86,0.320000,0.780000,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55182,6543262,LGMsIuesuDdK3lH3dM-dWA,3PYDOk5Us28fuHZAUEZq5g,ejzWpdWRZu4JM9tsZgZVEA,5.0,11,0,4,Wow. I had some gold rings/necklaces to sell a...,2014-06-02 21:08:24,moderate useful,en,wow gold ringsnecklaces sell took gold buyer c...,80.92,0.164017,0.426303,0.150943
55183,5441853,K7oX_4ZXoHxFAJV8IBDC0w,vpoRmKwfFDW3hj5baBNdiA,ggqr9BEklylICKl770GDlQ,1.0,1,1,1,Use to love to go here at least once a month. ...,2017-06-13 07:59:41,not useful,en,use love go least month last couple time food ...,102.61,-0.045833,0.602778,0.176471
55184,1415430,ojJGvcMZObHdHrvvm__Qlw,yROiz_5pQose97qyETplzQ,ICtCTow-58Ie24hOC1eG3w,5.0,0,0,0,"They got tripe, they got beef tongue, they got...",2017-06-05 23:49:32,not useful,en,got tripe got beef tongue got cheapest shrimp ...,91.11,0.263803,0.544530,0.276923
55185,5602623,sFQ8waEtUsguHfAgbxhGJA,AkBZo05E0KApriZqxXgfRQ,x245eNNdQ6i4FbZsDeCdJw,5.0,13,28,15,"Great bar, or greatest bar? \n\nMaybe it's my ...",2011-11-21 22:49:25,moderate useful,en,great bar greatest bar maybe blue collar root ...,73.71,0.191353,0.491893,0.185841


In [12]:
newReview = Review_project.merge(NewUser,on='user_id',how='left')

In [13]:
newReview['NumofWords'] = newReview['text'].str.split().str.len()

In [14]:
newReview['NumElite'] = newReview['NumElite'].fillna(0).astype(int)

In [15]:
newReview.to_csv('Review_project_sentiment_wUser.csv', sep=',', index=False)

In [16]:
Review_project_wUser=pd.read_csv('Review_project_sentiment_wUser.csv')

In [17]:
Review_project_wUser['useful_level']

0        moderate useful
1        moderate useful
2             not useful
3                 useful
4             not useful
              ...       
55182    moderate useful
55183         not useful
55184         not useful
55185    moderate useful
55186    moderate useful
Name: useful_level, Length: 55187, dtype: object

In [18]:
Review_project_wUser1=Review_project_wUser
Review_project_wUser1.loc[Review_project_wUser['useful_level']=="useful",'useful_level']=2
Review_project_wUser1.loc[Review_project_wUser['useful_level']=="moderate useful",'useful_level']=1
Review_project_wUser1.loc[Review_project_wUser['useful_level']=="not useful",'useful_level']=0

In [19]:
Review_project_wUser1.to_csv('Review_project_sentiment_wUser.csv', sep=',', index=False)